In [ ]:
import json
import datetime
from matplotlib import pyplot as plt
import pandas as pd
import requests

# Write functions

In [ ]:
def gen_payload(date, origin, avg_speed, max_wind_gust): 
    date = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%SZ')
    return [{
        "id": "urn:ngsi-ld:Impetus:windSensor:Manresa",
        "type": "WindSensor",
        "observed_at": {
            "type": "Property",
            "value": date,
            "observedAt": date,
        },      
        "origin": {
            "type": "Property",
            "value": origin,
            "observedAt": date,
        },
        "avg_speed":{
            "type": "Property",
            "value": avg_speed*1.0,
            "observedAt": date,
        },
        "max_wind_gust": {
            "type": "Property",
            "value": max_wind_gust*1.0,
            "observedAt": date,
        }        
        ,"@context": ["https://data-manager.climate-impetus.eu/schemas/WindSensor/schema.jsonld"]
    }]

def send_data(data):
    url_local = 'http://impetus-orion:1026/ngsi-ld/v1/entityOperations/upsert'
    url_remote = 'https://data-manager.climate-impetus.eu/broker/ngsi-ld/v1/entityOperations/upsert'
    response = requests.post(url=url_remote, headers={"content-type": "application/ld+json"},data=json.dumps(data))
    print(response.status_code)
    return response

# Read functions

In [ ]:
def generate_read_payload(start_time, end_time):
    url = "https://data-manager.climate-impetus.eu/access/historical/entities"
    params = {
        'start_time': start_time,
        'end_time': end_time,
        'entity_type': 'windsensor',
        'format': 'COLUMN'
    }
    headers = {'accept': 'application/json'}
    response = requests.get(url, params=params, headers=headers)
    return response # print(response.json())

# Send

## Send one

In [ ]:
date = '1970-01-01 00:00:00'
origin = "PROCESSED" #"DEVICE"
avg_speed = 99.0
max_wind_gust = 99.0
data = gen_payload(date, origin, avg_speed, max_wind_gust)
send_data(data)

## Send two

In [ ]:
origin = "PROCESSED" #"DEVICE"

date_1 = '1970-01-01 00:00:00'
avg_speed_1 = 5.0
max_wind_gust_1 = 8.0

date_2 = '1970-01-02 00:00:00'
avg_speed_2 = 10.0
max_wind_gust_2 = 20.0

data_1 = gen_payload(date_1, origin, avg_speed_1, max_wind_gust_1)
data_2 = gen_payload(date_2, origin, avg_speed_2, max_wind_gust_2)
payloads = [data_1[0], data_2[0]]
payloads
# print(payloads[0] == payloads[1])
# response = send_data(payloads)
response = send_data(data_1)
response = send_data(data_2)

In [ ]:
response.text

## Send dataframe

In [ ]:
df_payload

In [ ]:
df_payload = pd.DataFrame({"avg_speed":df_response.avg_speed, "max_wind_gust":df_response.max_wind_gust}, index=df_response.index)
date_list = df_payload.index.strftime('%Y-%m-%d %H:%M:%S')
max_wind_gust_list = [99]*len(df_payload.max_wind_gust.values)
avg_speed_list = df_payload.avg_speed.values
payloads_df = [ gen_payload(date_list[i],"PROCESSED", avg_speed_list[i] ,max_wind_gust_list[i])[0] for i in range(len(date_list))]
# payloads_df
# for payload in payloads_df: send_data([payload])
# send_data(payloads_df)

In [ ]:
def send_data_with_session(data, session):
    url_local = 'http://impetus-orion:1026/ngsi-ld/v1/entityOperations/upsert'
    url_remote = 'https://data-manager.climate-impetus.eu/broker/ngsi-ld/v1/entityOperations/upsert'
    response = session.post(url=url_remote, headers={"content-type": "application/ld+json"},data=json.dumps(data))
    print(response.status_code)
    return response

with requests.Session() as session:
    for payload in payloads_df:
        send_data_with_session([payload], session)

In [ ]:
payloads
payloads_df
print(payloads[1] == payloads_df[1])

#  Read

In [ ]:
response = generate_read_payload( "2017-01-01T00:00:00", "2017-02-01T00:00:00" )
# response.json()
df_response = parse_api_response_to_df(response.json())
# df_response.plot()

In [ ]:
df_response

In [ ]:
response = generate_read_payload( df_payload.sort_index().index[0],  df_payload.sort_index().index[-1])
# response.json()
df_response = parse_api_response_to_df(response.json())
df_response

In [ ]:
response.json()['urn:ngsi-ld:Impetus:windSensor:Manresa']['origin']

In [ ]:
response = generate_read_payload( '1970-01-01T00:00:00',  '1971-01-02T00:00:00')
response.json()
# print(json.dumps(response.json(), indent=2))

In [ ]:
def parse_api_response_to_df(payload):
    root = payload['urn:ngsi-ld:Impetus:windSensor:Manresa']
    column_names = [key for key in root.keys() if key not in ["date_observed", "entity_type", "origin"]]
    df = pd.DataFrame(root)
    df.loc[0, "avg_speed"] = 1
    df.date_observed = pd.to_datetime( df.date_observed, format="%Y-%m-%dT%H:%M:%S")
    df = df.set_index( "date_observed", drop=True )
    df = df[~df.index.duplicated( keep="last") ]
    return df

df_response = parse_api_response_to_df(response.json())

In [ ]:
df_response

# Read and plot

In [ ]:
# response = generate_read_payload( '1970-01-01T00:00:00',  '2020-01-01T00:00:00')
response = generate_read_payload( '2019-01-01T00:00:00',  '2020-01-01T00:00:00')
response.json()
print(json.dumps(response.json(), indent=2))
r = response.json()

In [ ]:
r

In [ ]:
avg_speed = r['urn:ngsi-ld:Impetus:windSensor:Manresa']['avg_speed']
max_wind_gust = r['urn:ngsi-ld:Impetus:windSensor:Manresa']['max_wind_gust']
date_observed = r['urn:ngsi-ld:Impetus:windSensor:Manresa']['date_observed']

manresa_df = pd.DataFrame({"avg_speed":avg_speed, "max_wind_gust":max_wind_gust}, index=date_observed)
manresa_df.index = pd.to_datetime(manresa_df.index)
manresa_df = manresa_df.sort_index()[4:]
_ = plt.plot(manresa_df[['avg_speed', 'max_wind_gust']])

In [ ]:
r = response.json()
df = pd.DataFrame(r['urn:ngsi-ld:Impetus:windSensor:Manresa'])
df.date_observed = pd.to_datetime( df.date_observed )
df = df.set_index("date_observed").drop("entity_type", axis=1)
df_device = df[ df.origin == "DEVICE"].drop("origin", axis=1)
df_processed = df[ df.origin == "PROCESSED"].drop("origin", axis=1)


plt.figure(figsize=(12,6))
_ = plt.plot(df_device[['max_wind_gust']], label="Original")
_ = plt.plot(df_processed[['max_wind_gust']], label="Processed")
_ = plt.legend()

In [ ]:
df_device = df[ df.origin == "DEVICE"].drop("origin", axis=1)
df_device = df_device["2015-03-01 00:00:00":"2015-07-15 00:00:00" ]
df_device[:60]

In [ ]:
r = response.json()
df = pd.DataFrame(r['urn:ngsi-ld:Impetus:windSensor:Manresa'])
df.date_observed = pd.to_datetime( df.date_observed )
df = df.set_index("date_observed").drop("entity_type", axis=1)
df_device = df[ df.origin == "DEVICE"].drop("origin", axis=1)
df_processed = df[ df.origin == "PROCESSED"].drop("origin", axis=1)


plt.figure(figsize=(12,6))
_ = plt.plot(df_device[['max_wind_gust']], label="Original")
_ = plt.plot(df_processed[['max_wind_gust']], label="Processed")
_ = plt.legend()